# Lambda School Data Science Module 142
## Sampling, Confidence Intervals, and Hypothesis Testing

In [1]:
import numpy as np

np.random.seed(42)

np.random.normal(size=10)

array([ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986, -0.23415337,
       -0.23413696,  1.57921282,  0.76743473, -0.46947439,  0.54256004])

## Prepare - examine other available hypothesis tests

If you had to pick a single hypothesis test in your toolbox, t-test would probably be the best choice - but the good news is you don't have to pick just one! Here's some of the others to be aware of:

In [2]:

from scipy.stats import chisquare  # One-way chi square test

# Chi square can take any crosstab/table and test the independence of rows/cols
# The null hypothesis is that the rows/cols are independent -> low chi square
# The alternative is that there is a dependence -> high chi square
# Be aware! Chi square does *not* tell you direction/causation

ind_obs = np.array([[1, 1], [2, 2]]).T
print(ind_obs)
print(chisquare(ind_obs, axis=None))

dep_obs = np.array([[16, 18, 16, 14, 12, 12], [32, 24, 16, 28, 20, 24]]).T
print(dep_obs)
print(chisquare(dep_obs, axis=None))

[[1 2]
 [1 2]]
Power_divergenceResult(statistic=0.6666666666666666, pvalue=0.8810148425137847)
[[16 32]
 [18 24]
 [16 16]
 [14 28]
 [12 20]
 [12 24]]
Power_divergenceResult(statistic=23.31034482758621, pvalue=0.015975692534127565)


In [3]:
# Distribution tests:
# We often assume that something is normal, but it can be important to *check*

# For example, later on with predictive modeling, a typical assumption is that
# residuals (prediction errors) are normal - checking is a good diagnostic

from scipy.stats import normaltest
# Poisson models arrival times and is related to the binomial (coinflip)
sample = np.random.poisson(5, 1000)
print(normaltest(sample))  # Pretty clearly not normal

NormaltestResult(statistic=22.774697660689363, pvalue=1.1338020541271413e-05)


In [4]:
# Kruskal-Wallis H-test - compare the median rank between 2+ groups
# Can be applied to ranking decisions/outcomes/recommendations
# The underlying math comes from chi-square distribution, and is best for n>5
from scipy.stats import kruskal

x1 = [1, 3, 5, 7, 9]
y1 = [2, 4, 6, 8, 10]
print(kruskal(x1, y1))  # x1 is a little better, but not "significantly" so

x2 = [1, 1, 1]
y2 = [2, 2, 2]
z = [2, 2]  # Hey, a third group, and of different size!
print(kruskal(x2, y2, z))  # x clearly dominates

KruskalResult(statistic=0.2727272727272734, pvalue=0.6015081344405895)
KruskalResult(statistic=7.0, pvalue=0.0301973834223185)


In [5]:
# https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test

And there's many more! `scipy.stats` is fairly comprehensive, though there are even more available if you delve into the extended world of statistics packages. As tests get increasingly obscure and specialized, the importance of knowing them by heart becomes small - but being able to look them up and figure them out when they *are* relevant is still important.

## Live Lecture - let's explore some more of scipy.stats

Candidate topics to explore:

- `scipy.stats.chi2` - the Chi-squared distribution, which we can use to reproduce the Chi-squared test
- Calculate the Chi-Squared test statistic "by hand" (with code), and feed it into `chi2`
- Build a confidence interval with `stats.t.ppf`, the t-distribution percentile point function (the inverse of the CDF) - we can write a function to return a tuple of `(mean, lower bound, upper bound)` that you can then use for the assignment (visualizing confidence intervals)

In [48]:
import pandas as pd

df = pd.read_csv('/Users/imac/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/module2-sampling-confidence-intervals-and-hypothesis-testing/house-votes-84.data', sep=',')
df.head()

,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [42]:
df.isnull().sum()

republican    0
n             0
y             0
n.1           0
y.1           0
y.2           0
y.3           0
n.2           0
n.3           0
n.4           0
y.4           0
?             0
y.5           0
y.6           0
y.7           0
n.5           0
y.8           0
dtype: int64

In [49]:
table_observed = pd.crosstab(df['n'], df['y'])
table_observed

y,?,n,y
n,,,
?,8,2,2
n,23,108,104
y,17,82,88


In [50]:
table_observed_w_marg = pd.crosstab(df['n'], df['y'], margins=True)
table_observed_w_marg.head()

y,?,n,y,All
n,,,,
?,8,2,2,12
n,23,108,104,235
y,17,82,88,187
All,48,192,194,434


In [51]:
table_observed_w_marg['All']

n
?       12
n      235
y      187
All    434
Name: All, dtype: int64

In [52]:
table_observed_w_marg.T['All']

y
?       48
n      192
y      194
All    434
Name: All, dtype: int64

In [53]:
table_observed_w_marg['All'].values[:-1]

array([ 12, 235, 187])

In [54]:
np.outer(table_observed_w_marg['All'].values[:-1], table_observed_w_marg.T['All'].values[:-1])/table_observed_w_marg.T['All'].values[-1]

array([[  1.32718894,   5.30875576,   5.3640553 ],
       [ 25.99078341, 103.96313364, 105.04608295],
       [ 20.68202765,  82.7281106 ,  83.58986175]])

In [55]:
goals_totals = table_observed_w_marg['All'].values[:-1]
goals_totals

array([ 12, 235, 187])

In [56]:
grade_totals = table_observed_w_marg.T['All'].values[:-1]
grade_totals

array([ 48, 192, 194])

In [57]:
assert goals_totals.sum() == grade_totals.sum()

In [58]:
grade_totals.sum()

434

In [59]:
expected = np.outer(goals_totals, grade_totals)/grade_totals.sum()
expected

array([[  1.32718894,   5.30875576,   5.3640553 ],
       [ 25.99078341, 103.96313364, 105.04608295],
       [ 20.68202765,  82.7281106 ,  83.58986175]])

In [60]:
table_expected = pd.DataFrame(expected)
table_expected.columns = table_observed.columns
table_expected.index = table_observed.index
table_expected

y,?,n,y
n,,,
?,1.327189,5.308756,5.364055
n,25.990783,103.963134,105.046083
y,20.682028,82.728111,83.589862


In [ ]:
table_observed

In [ ]:
deviation = table_observed - table_expected
deviation

In [ ]:
deviation_sq = deviation**2
deviation_sq

In [ ]:
fraction = deviation_sq/table_expected
fraction

In [ ]:
fraction.values.sum()

In [ ]:
table_observed

In [ ]:
from scipy.stats import chi2_contingency

chi2, p, df, expect = chi2_contingency(table_observed)
print(f'chi2 stat: {chi2}')
print(f'p-value: {p}')
print(f'df: {df}')
print(f'expected freq: {expect}')

# Confidence intervals

In [ ]:
import scipy.stats as stats
import numpy as np

#confidence_interval = [lower_bound, upper_bound]

coinflips = np.random.binomial(n=1, p=.501, size=10000000)
print(coinflips[:20])

In [ ]:
stats.ttest_1samp(coinflips, 0.5)

In [ ]:
coinflips_100 = np.random.binomial(n=1, p=.5, size=100)
coinflips_1000 = np.random.binomial(n=1, p=.5, size=1000)

print("100 Coinflips Standard Deviation:", np.std(coinflips_100, ddof=1))
print("1000 Coinflips Standard Deviation:", np.std(coinflips_1000, ddof=1))

In [ ]:
def my_stdev(sample_data):
  mu = sample_data.mean()
  diff = sample_data - mu
  diff_sq = diff**2
  stdev = np.sqrt(diff_sq.sum()/(len(sample_data)-1))
  return stdev

In [ ]:
delta_100 = my_stdev(coinflips_100)/np.sqrt(100)
delta_100

In [ ]:
delta_1000 = my_stdev(coinflips_1000)/np.sqrt(1000)
delta_1000

In [ ]:
coinflips_1000.mean() - delta_1000

In [ ]:
coinflips_1000.mean() + delta_1000

In [ ]:
coinflips_100.mean() - delta_100

In [ ]:
coinflips_100.mean() + delta_100

## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):

1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?

Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.
3. Refactor your code so it is elegant, readable, and can be easily run for all issues.

In [13]:
# Importing libraries 

import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import scipy as sci

In [14]:
# DataFrame Variables for a local file with header names
# the filepath to the file should be point to its place on your HD 

file  = '/Users/imac/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/module2-sampling-confidence-intervals-and-hypothesis-testing/house-votes-84.data'
names = [
'Class',
'handicapped-infants',
'water-project-cost-sharin',
'adoption-of-the-budget-resolution',
'physician-fee-freeze',
'el-salvador-aid',
'religious-groups-in-schools',
'anti-satellite-test-ban',
'aid-to-nicaraguan-contras',
'mx-missile',
'immigration',
'synfuels-corporation-cutback',
'education-spending',
'superfund-right-to-sue',
'crime',
'duty-free-exports',
'export-administration-act-south-africa']


In [15]:
# Reading in a file on local HD 

df = pd.read_csv(file, header=None, names=names)

In [16]:
df.head()

,Class,handicapped-infants,water-project-cost-sharin,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [17]:
# Checking for missing values
df.isnull().sum()

Class                                     0
handicapped-infants                       0
water-project-cost-sharin                 0
adoption-of-the-budget-resolution         0
physician-fee-freeze                      0
el-salvador-aid                           0
religious-groups-in-schools               0
anti-satellite-test-ban                   0
aid-to-nicaraguan-contras                 0
mx-missile                                0
immigration                               0
synfuels-corporation-cutback              0
education-spending                        0
superfund-right-to-sue                    0
crime                                     0
duty-free-exports                         0
export-administration-act-south-africa    0
dtype: int64

In [18]:
# Creating a crosstab of the DataFrame 
xtab = pd.crosstab(df['handicapped-infants'], df['water-project-cost-sharin'])
xtab

water-project-cost-sharin,?,n,y
handicapped-infants,,,
?,8,2,2
n,23,108,105
y,17,82,88


In [19]:
# Adding margins to crosstable

xtab_marg = pd.crosstab(df['handicapped-infants'], df['water-project-cost-sharin'] , margins=True)
xtab_marg.head()

water-project-cost-sharin,?,n,y,All
handicapped-infants,,,,
?,8,2,2,12
n,23,108,105,236
y,17,82,88,187
All,48,192,195,435


In [20]:
# Creating a numpy array of of the 2 features 

np.outer(xtab_marg['All'].values[:-1],
xtab_marg.T['All'].values[:-1])/xtab_marg.T['All'].values[-1]

array([[  1.32413793,   5.29655172,   5.37931034],
       [ 26.04137931, 104.16551724, 105.79310345],
       [ 20.63448276,  82.53793103,  83.82758621]])

In [21]:
# Selecting summed values of the first feature 

xtab_marg['All']

handicapped-infants
?       12
n      236
y      187
All    435
Name: All, dtype: int64

In [22]:
# Selecting summed values of the second feature 
xtab_marg.T['All']

water-project-cost-sharin
?       48
n      192
y      195
All    435
Name: All, dtype: int64

In [23]:
# Creating a np array of first feature 

xtab_marg['All'].values[:-1]

array([ 12, 236, 187])

In [24]:
# Creating a np array of the second feature 

xtab_marg.T['All'].values[:-1]

array([ 48, 192, 195])

In [25]:
# Assigning np array to a variable 

handi_total = xtab_marg.values[:-1]

In [26]:
# Assigning transposed np array to a variable 

water_total = xtab_marg.T.values[:-1]

In [27]:
# Verifying the arrays are the same size 

assert handi_total.sum() == water_total.sum()

In [28]:
# Verifying the size of the new feature

water_total.sum()

870

In [35]:
expected = np.outer(handi_total, water_total)/handi_total.sum()

In [37]:
expected.T

array([[7.35632184e-02, 1.83908046e-02, 1.83908046e-02, 1.10344828e-01,
        2.11494253e-01, 9.93103448e-01, 9.65517241e-01, 2.17011494e+00,
        1.56321839e-01, 7.54022989e-01, 8.09195402e-01, 1.71954023e+00],
       [2.11494253e-01, 5.28735632e-02, 5.28735632e-02, 3.17241379e-01,
        6.08045977e-01, 2.85517241e+00, 2.77586207e+00, 6.23908046e+00,
        4.49425287e-01, 2.16781609e+00, 2.32643678e+00, 4.94367816e+00],
       [1.56321839e-01, 3.90804598e-02, 3.90804598e-02, 2.34482759e-01,
        4.49425287e-01, 2.11034483e+00, 2.05172414e+00, 4.61149425e+00,
        3.32183908e-01, 1.60229885e+00, 1.71954023e+00, 3.65402299e+00],
       [4.41379310e-01, 1.10344828e-01, 1.10344828e-01, 6.62068966e-01,
        1.26896552e+00, 5.95862069e+00, 5.79310345e+00, 1.30206897e+01,
        9.37931034e-01, 4.52413793e+00, 4.85517241e+00, 1.03172414e+01],
       [1.83908046e-02, 4.59770115e-03, 4.59770115e-03, 2.75862069e-02,
        5.28735632e-02, 2.48275862e-01, 2.41379310e-01, 5.42

In [40]:
tab_expected = pd.DataFrame(expected)
tab_expected.columns = xtab.columns
tab_expected.index = xtab.index
tab_expected

ValueError: Length mismatch: Expected axis has 12 elements, new values have 3 elements

In [61]:
library(ggplot2)
alpha <- 0.05 # a significance level
n <- 10       # degree of freedom
accept <- function(x){
  y <- dt(x, n)
  y[x < qt(alpha/2, n) | x > qt(1 - alpha/2, n)] <- NA
  return(y)
}
reject <- function(x){
  y <- dt(x, n)
  y[x > qt(alpha/2, n) & x < qt(1 - alpha/2, n)] <- NA
  return(y)
}
N <- 1e4 # a large number
s <- 5   # fontsize
p <-
  ggplot(data.frame(x=c(-3, +3)), aes(x=x)) +
  stat_function(
    fun=accept, geom="area",
    fill="#0072B2", n=N) + 
  stat_function(
    fun=reject, geom="area",
    fill="#D55E00", n=N) +
  annotate(
    "text", x=0, y=0.2, 
    parse=T, label="1-alpha", 
    size=s, colour="#EEEEEE") +  
  annotate(
    "text", x=-2.5, y=0.2, 
    parse=T, label="alpha/2", 
    size=s, colour="#D55E00") +
  annotate(
    "text", x=+2.5, y=0.2, 
    parse=T, label="alpha/2", 
    size=s, colour="#D55E00") +
  annotate(
    "text", x=qt(alpha/2, n), y=-0.02,
    parse=T, label="-t[n-1](alpha/2)",
    size=s, colour="#000000") +
  annotate(
    "text", x=0, y=-0.02,
    label="0",
    size=s, colour="#000000") +
  annotate(
    "text", x=qt(1 - alpha/2, n), y=-0.02,
    parse=T, label="+t[n-1](alpha/2)",
    size=s, colour="#000000") +
  theme(
    axis.text.y=element_blank(), 
    axis.text.x=element_blank(),
    axis.ticks=element_blank()) +
  labs(x="", y="")

svg(filename="confidence-interval.svg", width=16/2, height=9/2)
plot(p)
dev.off()

SyntaxError: invalid syntax (<ipython-input-61-aa4f486054b3>, line 4)

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)